In [ ]:
import pandas as pd 
import numpy as np
import tensorflow as tf

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.cm as cm

In [ ]:
data = pd.read_csv("data.csv")

In [ ]:
#check the number of images and each image data variable
data.shape

In [ ]:
data.head()

In [ ]:
np.unique(data["Usage"].values.ravel())

In [ ]:
print('The number of training data set is %d'%(len(data[data.Usage == "Training"])))

In [ ]:
train_data = data[data.Usage == "Training"]

In [ ]:
pixels_values = train_data.pixels.str.split(" ").tolist()

In [ ]:
pixels_values = pd.DataFrame(pixels_values, dtype=int)

In [ ]:
print(pixels_values)

In [ ]:
images = pixels_values.values

In [ ]:
print(images)

In [ ]:
images = images.astype(np.float)

In [ ]:
images

In [ ]:
#Define a function to show image through 48*48 pixels
def show(img):
    show_image = img.reshape(48,48)
    
    #plt.imshow(show_image, cmap=cm.binary)
    plt.imshow(show_image, cmap='gray')

In [ ]:
#show one image
show(images[2])

In [ ]:
print(images.mean(axis=1))

In [ ]:
images = images - images.mean(axis=1).reshape(-1,1)

In [ ]:
show(images[2])

In [ ]:
images = np.multiply(images,100.0/255.0)

In [ ]:
each_pixel_mean = images.mean(axis=0)

In [ ]:
each_pixel_std = np.std(images, axis=0)

In [ ]:
images = np.divide(np.subtract(images,each_pixel_mean), each_pixel_std)

In [ ]:
print(images.shape[1])

In [ ]:
image_pixels = images.shape[1]
print ('Flat pixel values is %d'%(image_pixels))

In [ ]:
image_width = image_height = np.ceil(np.sqrt(image_pixels)).astype(np.uint8)

In [ ]:
image_width

In [ ]:
#print(labels_flat.shape)

In [ ]:
labels_flat = train_data["emotion"].values.ravel()

In [ ]:
labels_count = np.unique(labels_flat).shape[0]

In [ ]:
print('The number of different facial expressions is %d'%labels_count)

In [ ]:
def dense_to_one_hot(labels_dense, num_classes):
    print(labels_dense)
    print(num_classes)
    num_labels = labels_dense.shape[0]
    print(num_labels)
    index_offset = np.arange(num_labels) * num_classes
    print(index_offset)
    labels_one_hot = np.zeros((num_labels, num_classes))
    print(labels_one_hot)
    labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
    print(labels_one_hot)
    return labels_one_hot

In [ ]:
labels = dense_to_one_hot(labels_flat, labels_count)
labels = labels.astype(np.uint8)

In [ ]:
labels[2]

In [ ]:
# split data into training & validation
VALIDATION_SIZE = 1709

In [ ]:
validation_images = images[:VALIDATION_SIZE]
validation_labels = labels[:VALIDATION_SIZE]

train_images = images[VALIDATION_SIZE:]
train_labels = labels[VALIDATION_SIZE:]

In [ ]:
print('The number of final training data: %d'%(len(train_images)))

In [ ]:
#BUILDING TENSORFLOW CNN MODEL
# weight initialization
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=1e-4)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [ ]:
# convolution
def conv2d(x, W, padd):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding=padd)

In [ ]:
# pooling
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')

In [ ]:
# input & output of NN
# images
x = tf.placeholder('float', shape=[None, image_pixels])
# labels
y_ = tf.placeholder('float', shape=[None, labels_count])

In [ ]:
print(x)
print(y_)

In [ ]:
# first convolutional layer 64
W_conv1 = weight_variable([5, 5, 1, 64])
b_conv1 = bias_variable([64])

# (27000, 2304) => (27000,48,48,1)
image = tf.reshape(x, [-1,image_width , image_height,1])
#print (image.get_shape()) # =>(27000,48,48,1)


h_conv1 = tf.nn.relu(conv2d(image, W_conv1, "SAME") + b_conv1)
#print (h_conv1.get_shape()) # => (27000,48,48,64)
h_pool1 = max_pool_2x2(h_conv1)
#print (h_pool1.get_shape()) # => (27000,24,24,1)
h_norm1 = tf.nn.lrn(h_pool1, 4, bias=1.0, alpha=0.001/9.0, beta=0.75)

In [ ]:
# second convolutional layer
W_conv2 = weight_variable([5, 5, 64, 128])
b_conv2 = bias_variable([128])

h_conv2 = tf.nn.relu(conv2d(h_norm1, W_conv2, "SAME") + b_conv2)
#print (h_conv2.get_shape()) # => (27000,24,24,128)

h_norm2 = tf.nn.lrn(h_conv2, 4, bias=1.0, alpha=0.001/9.0, beta=0.75)

h_pool2 = max_pool_2x2(h_norm2)

In [ ]:
# local layer weight initialization
def local_weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.04)
    return tf.Variable(initial)

def local_bias_variable(shape):
    initial = tf.constant(0.0, shape=shape)
    return tf.Variable(initial)

In [ ]:
# densely connected layer local 3
W_fc1 = local_weight_variable([12 * 12 * 128, 3072])
b_fc1 = local_bias_variable([3072])

# (27000, 12, 12, 128) => (27000, 12 * 12 * 128)
h_pool2_flat = tf.reshape(h_pool2, [-1, 12 * 12 * 128])

h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
#print (h_fc1.get_shape()) # => (27000, 1024)???

In [ ]:
# densely connected layer local 4
W_fc2 = local_weight_variable([3072, 1536])
b_fc2 = local_bias_variable([1536])

# (27000, 12, 12, 128) => (27000, )
h_fc2_flat = tf.reshape(h_fc1, [-1, 3072])

h_fc2 = tf.nn.relu(tf.matmul(h_fc2_flat, W_fc2) + b_fc2)
#print (h_fc1.get_shape()) 

In [ ]:
# dropout
keep_prob = tf.placeholder('float')
h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)

In [ ]:
# readout layer for deep net
W_fc3 = weight_variable([1536, labels_count])
b_fc3 = bias_variable([labels_count])

y = tf.nn.softmax(tf.matmul(h_fc2_drop, W_fc3) + b_fc3)

#print (y.get_shape()) # 

In [ ]:
# settings
LEARNING_RATE = 1e-4

In [ ]:
# cost function
cross_entropy = -tf.reduce_sum(y_*tf.log(y))


# optimisation function
train_step = tf.train.AdamOptimizer(LEARNING_RATE).minimize(cross_entropy)

# evaluation
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))

In [ ]:
# prediction function
#[0.1, 0.9, 0.2, 0.1, 0.1 0.3, 0.5, 0.1, 0.2, 0.3] => 1
predict = tf.argmax(y,1)

In [ ]:
# set to 10000 iterations 
TRAINING_ITERATIONS = 10000
    
DROPOUT = 0.6
BATCH_SIZE = 50

In [ ]:
epochs_completed = 0
index_in_epoch = 0
num_examples = train_images.shape[0]

# serve data by batches
def next_batch(batch_size):
    
    global train_images
    global train_labels
    global index_in_epoch
    global epochs_completed
    
    start = index_in_epoch
    index_in_epoch += batch_size
    
    # when all trainig data have been already used, it is reorder randomly    
    if index_in_epoch > num_examples:
        # finished epoch
        epochs_completed += 1
        # shuffle the data
        perm = np.arange(num_examples)
        np.random.shuffle(perm)
        train_images = train_images[perm]
        train_labels = train_labels[perm]
        # start next epoch
        start = 0
        index_in_epoch = batch_size
        assert batch_size <= num_examples
    end = index_in_epoch
    return train_images[start:end], train_labels[start:end]

In [ ]:
# start TensorFlow session
init = tf.initialize_all_variables()
sess = tf.InteractiveSession()

sess.run(init)

In [ ]:
# visualisation variables
train_accuracies = []
validation_accuracies = []
x_range = []

display_step=1

In [ ]:
for i in range(TRAINING_ITERATIONS):

    #get new batch
    batch_xs, batch_ys = next_batch(BATCH_SIZE)        

    # check progress on every 1st,2nd,...,10th,20th,...,100th... step
    if i%display_step == 0 or (i+1) == TRAINING_ITERATIONS:
        
        train_accuracy = accuracy.eval(feed_dict={x:batch_xs, 
                                                  y_: batch_ys, 
                                                  keep_prob: 1.0})       
        if(VALIDATION_SIZE):
            validation_accuracy = accuracy.eval(feed_dict={ x: validation_images[0:BATCH_SIZE], 
                                                            y_: validation_labels[0:BATCH_SIZE], 
                                                            keep_prob: 1.0})                                  
            print('training_accuracy / validation_accuracy => %.2f / %.2f for step %d'%(train_accuracy, validation_accuracy, i))
            
            validation_accuracies.append(validation_accuracy)
            
        else:
             print('training_accuracy => %.4f for step %d'%(train_accuracy, i))
        train_accuracies.append(train_accuracy)
        x_range.append(i)
        
        # increase display_step
        if i%(display_step*10) == 0 and i and display_step<100:
            display_step *= 10
    # train on batch
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys, keep_prob: DROPOUT})

In [ ]:
#RESULT ANALYSIS
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import itertools

In [ ]:
# check final accuracy on validation set  
if(VALIDATION_SIZE):
    validation_accuracy = accuracy.eval(feed_dict={x: validation_images, 
                                                   y_: validation_labels, 
                                                   keep_prob: 1.0})
    print('validation_accuracy => %.4f'%validation_accuracy)
    plt.plot(x_range, train_accuracies,'-b', label='Training')
    plt.plot(x_range, validation_accuracies,'-g', label='Validation')
    plt.legend(loc='lower right', frameon=False)
    plt.ylim(ymax = 1.0, ymin = 0.0)
    plt.ylabel('accuracy')
    plt.xlabel('step')
    plt.show()

In [ ]:
saver = tf.train.Saver(tf.global_variables())

In [ ]:
#saver.save(sess, 'my-model1', global_step=0)

In [ ]:
# read test data from CSV file 
test_data = data[data.Usage == "PrivateTest"] #changed to privatetest

In [ ]:
test_data.head()
len(test_data)

In [ ]:
test_pixels_values = test_data.pixels.str.split(" ").tolist()

In [ ]:
test_pixels_values = pd.DataFrame(test_pixels_values, dtype=int)

In [ ]:
test_images = test_pixels_values.values

In [ ]:
test_images = test_images - test_images.mean(axis=1).reshape(-1,1)
test_images = np.multiply(test_images,100.0/255.0)

In [ ]:
test_images = np.divide(np.subtract(test_images,each_pixel_mean), each_pixel_std)

In [ ]:
print('test_images({0[0]},{0[1]})'.format(test_images.shape))

# predict test set
#predicted_lables = predict.eval(feed_dict={x: test_images, keep_prob: 1.0})

# using batches is more resource efficient
predicted_lables = np.zeros(test_images.shape[0])

In [ ]:
for i in range(0,test_images.shape[0]//BATCH_SIZE):
    predicted_lables[i*BATCH_SIZE : (i+1)*BATCH_SIZE] = predict.eval(feed_dict={x: test_images[i*BATCH_SIZE : (i+1)*BATCH_SIZE], 
                                                                                keep_prob: 1.0})


print('predicted_lables({0})'.format(len(predicted_lables)))

In [ ]:
predicted_lables

In [ ]:
test_data.emotion.values

In [ ]:
accuracy_score(test_data.emotion.values, predicted_lables)

In [ ]:
confusion_matrix(test_data.emotion.values, predicted_lables)

In [ ]:
cnf_matrix = confusion_matrix(test_data.emotion.values, predicted_lables)

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, int(cm[i, j]*100)/100.0,
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
class_names = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

In [ ]:
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=False,
                      title='Confusion Matrix for Test Dataset')

plt.show()